# Quantum Teleportation

In [5]:
using QuantumInformation
using StatsBase

In [6]:
"Projective measurement with standard basis"
function measure(state, n)
    nqubit = state.basis |> length |> log2 |> Int
    mops = [identityoperator(SpinBasis(1//2)) for i in 1:nqubit]
    mops[n] = spinup(SpinBasis(1//2)) |> dm |> sparse
    mop = tensor(mops...)
    p = real( trace( mop * dm(state) ) )

    wv = pweights([p, 1-p])
    outcome = sample(0:1, wv)
    if outcome == 0
        state = mop * state / sqrt(p) 
    else
        mops[n] = spindown(SpinBasis(1//2)) |> dm |> sparse
        mop = tensor(mops...)
        state = mop * state / sqrt(1-p)
    end
    return outcome, state
end

measure

# Circuit of Quantum Teleportation

![teleport](./teleport.png)

In [7]:
basis = SpinBasis(1//2)
θ = π / 3 
α, β = cos(θ), sin(θ)
id = identityoperator(basis)

println("Initial state")
@show ψ = α * spinup(basis) + β * spindown(basis)
states = ψ ⊗ spinup(basis) ⊗ spinup(basis)

states = (id ⊗ hadamard() ⊗ id) * states
states = (id ⊗ cnot()) * states
states = (cnot() ⊗ id) * states
states = (hadamard() ⊗ id ⊗ id) * states
outcome1, states = measure(states, 1)
outcome2, states = measure(states, 2)
finstate = ptrace(states, [1, 2]) # density operator

println("\nBefore applying X and Z gates")
@show finstate

finstate = sigmaz()^outcome1 * sigmax()^outcome2 * finstate * sigmax()^outcome2 * sigmaz()^outcome1

println("\nMeasurement outcome")
@show outcome1, outcome2

println("\nInitial state |ψ><ψ|")
@show dm(ψ)

println("\nTeleported state")
@show finstate

println("\nCan we teleport quantum state?: ", dm(ψ).data ≈ finstate.data)

Initial state
ψ = α * spinup(basis) + β * spindown(basis) = Ket(dim=2)
  basis: Spin(1/2)
      0.5+0.0im
 0.866025+0.0im

Before applying X and Z gates
finstate = DenseOperator(dim=2x2)
  basis: Spin(1/2)
      0.25+0.0im  -0.433013+0.0im
 -0.433013+0.0im       0.75+0.0im

Measurement outcome
(outcome1, outcome2) = (1, 0)

Initial state |ψ><ψ|
dm(ψ) = DenseOperator(dim=2x2)
  basis: Spin(1/2)
     0.25+0.0im  0.433013+0.0im
 0.433013+0.0im      0.75+0.0im

Teleported state
finstate = DenseOperator(dim=2x2)
  basis: Spin(1/2)
     0.25+0.0im  0.433013+0.0im
 0.433013+0.0im      0.75+0.0im

Can we teleport quantum state?: true


In [8]:
versioninfo()

Julia Version 0.6.2
Commit d386e40 (2017-12-13 18:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU           E5645  @ 2.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (DYNAMIC_ARCH NO_AFFINITY Nehalem)
  LAPACK: libopenblas
  LIBM: libm
  LLVM: libLLVM-3.9.1 (ORCJIT, westmere)
